<a href="https://colab.research.google.com/github/lucasgab2230/LuwiAI-V2-Coder/blob/main/src/data_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: Create a code that search the technologies docs URLs and they sub-URLs and download code snippets from URLs. However, it will use BeautifulSoup and Requests to search across the internet.

!pip install requests beautifulsoup4

import requests
from bs4 import BeautifulSoup
import time

def find_code_snippets(url):
    try:
        response = requests.get(url, timeout=10) # Added timeout for robustness
        response.raise_for_status() # Raise an exception for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')

        code_snippets = []
        # This is a very basic search for <pre> and <code> tags.
        # You would need much more sophisticated logic here.
        for pre_tag in soup.find_all('pre'):
            code_snippets.append(pre_tag.get_text())
        for code_tag in soup.find_all('code'):
             code_snippets.append(code_tag.get_text())

        return code_snippets

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return []
    except Exception as e:
        print(f"An error occurred processing {url}: {e}")
        return []

# Example usage (replace with actual tech documentation URLs)
# You would need to expand this to handle multiple URLs and crawl links.
initial_urls = [
    'https://docs.python.org/3/',
    'https://developer.mozilla.org/en-US/docs/Web/HTML', # HTML Docs
    'https://developer.mozilla.org/en-US/docs/Web/CSS',  # CSS Docs
    'https://developer.mozilla.org/en-US/docs/Web/JavaScript', # JS Docs
    'https://react.dev/reference/react', # React Docs
    'https://git-scm.com/doc', # Git Docs
    'https://www.typescriptlang.org/docs/' # TS Docs
    # Add more initial documentation URLs here
]

all_snippets = {}
visited_urls = set()
url_queue = initial_urls[:]

while url_queue:
    current_url = url_queue.pop(0)
    if current_url in visited_urls:
        continue

    print(f"Visiting: {current_url}")
    visited_urls.add(current_url)

    snippets = find_code_snippets(current_url)
    if snippets:
        all_snippets[current_url] = snippets
        # In a real application, you would save these snippets to files or a database

    # --- Basic link finding (needs significant improvement for real-world scraping) ---
    try:
        response = requests.get(current_url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        for link in soup.find_all('a', href=True):
            href = link['href']
            # Very basic check: only process absolute URLs for now and stay on the same domain
            if href.startswith('http') and requests.utils.urlparse(href).netloc == requests.utils.urlparse(current_url).netloc:
                 if href not in visited_urls and href not in url_queue:
                    url_queue.append(href)
    except Exception as e:
        print(f"Error finding links on {current_url}: {e}")
    # --- End of basic link finding ---

    # Add a small delay to be polite to the server
    time.sleep(1)

# Print some of the collected snippets
for url, snippets in list(all_snippets.items())[:5]: # Print snippets from the first 5 URLs
    print(f"\nCode Snippets from {url}:")
    for i, snippet in enumerate(snippets):
        print(f"Snippet {i+1}:\n{snippet}\n---")